In [1]:
from vnpy.trader.vtConstant import *
from vnpy.trader.app.ctaStrategy import CtaTemplate
import numpy as np
import talib as ta
from datetime import timedelta, datetime
import time

########################################################################
class adxMacdEosStrategy(CtaTemplate):
    className = 'adxMacdEosStrategy'
    author = 'ChannelCMT'

    # 策略参数
    barPeriod = 200
    adxPeriod = 15; adxMaType = 0; adxThreshold = 20
    fastPeriod = 30; slowPeriod = 65; signalPeriod = 30
    fastMaType = 0; slowMaType = 0; signalMaType = 3
    VWAPPeriod = 50
    RSIPeriod=20
    long=8

    # 风控参数
    trailingPct = 0.026; stopControlTime = 5
    takeProfitFirstPct = 0.04; takeProfitSecondPct = 0.065
    lot = 1800

    # 仓位管理参数
    addPct = 0.004; addMultipler = 1

    # 信号变量
    adxCanTrade = 0; priceDirection = 0
    stopLossControl = 0

    transactionPrice={}
    # 自维护仓位与订单
    ownPosDict = {}; orderDict = {}

    # 参数列表，保存了参数的名称
    paramList = [
                 'symbolList',
                 'adxPeriod', 'adxMaType', 'adxThreshold',
                 'fastPeriod', 'slowPeriod','signalPeriod',
                 'fastMaType', 'slowMaType', 'signalMaType',
                 'trailingPct', 'stopControlTime',
                 'addPct','addMultipler','lot',
                 'takeProfitFirstPct', 'takeProfitSecondPct',"long"
                 ]

    # 变量列表，保存了变量的名称
    varList = ['ownPosDict',
               'adxCanTrade', 'priceDirection',
               'transactionPrice',
               'stopLossControl'
               ]
    # 同步列表，保存了需要保存到数据库的变量名称
    syncList = ['posDict', 'eveningDict']

    # ----------------------------------------------------------------------
    def __init__(self, ctaEngine, setting):
        super().__init__(ctaEngine, setting)

    # ----------------------------------------------------------------------
    def onInit(self):
        """初始化策略"""
        self.writeCtaLog(u'策略初始化')
        # signalVar
        self.setArrayManagerSize(self.barPeriod)
        self.tickObject = None

        # riskControlVar
        self.transactionPrice = {s+'0': 0 for s in self.symbolList}
        self.closeTime = None
        self.longStop = 0
        self.shortStop = 999999
        self.intraTradeHighDict = 0
        self.intraTradeLowDict = 999999

        # posManage
        self.nPos = 0

        for s in self.symbolList:
            self.ownPosDict = {s + '_LONG': 0, s + '_SHORT': 0}
            self.orderDict = {s + '_OPEN': [], s + '_CLOSE': []}

        # 订单管理
        self.toExcuteOrders = {}
        self.toExcuteOrdersID = 0
        self.mail("chushihuaaaaaaaaaaaaaaaaaaaaaaaaa")
        self.putEvent()

    # ----------------------------------------------------------------------
    def onStart(self):
        """启动策略（必须由用户继承实现）"""
        self.writeCtaLog(u'策略启动')
        self.putEvent()

    # ----------------------------------------------------------------------
    def onStop(self):
        """停止策略（必须由用户继承实现）"""
        self.writeCtaLog(u'策略停止')
        self.putEvent()

    # ----------------------------------------------------------------------
    def onTick(self, tick):
        """收到行情TICK推送"""
        if not self.dataBlock(dataTime=tick.datetime, now=datetime.now(), maxDelay=5):
            engineType = self.getEngineType()
            if engineType == 'trading':
                self.tickObject = tick
            else:
                pass

    # 过滤掉实盘推数据可能产生的阻塞(5s延迟)
    def dataBlock(self, dataTime, now, maxDelay=5):
        if abs(now - dataTime).total_seconds() > maxDelay:
            self.writeCtaLog(
                "数据推送阻塞,跳过该次推送:now=%s,dataTime=%s" % (now.strftime("%Y-%m-%d %H:%M:%S"),
                                                      dataTime.strftime("%Y-%m-%d %H:%M:%S")))
            return True
        else:
            return False

    # orderManagement--------------------------------------------------------
    def cancelCloseOrder(self, bar):
        symbol = bar.vtSymbol
        haveCloseOrder = len(self.orderDict[symbol + '_CLOSE'])
        if haveCloseOrder:
            canceling = list(self.orderDict[symbol + '_CLOSE'])
            for closeOrderId in canceling:
                self.cancelOrder(closeOrderId)
            return False, canceling
        else:
            return True, []

    def priceExecute(self, bar):
        engineType = self.getEngineType()
        if engineType == 'trading':
            buyExecute = self.tickObject.upperLimit*0.99
            shortExecute = self.tickObject.lowerLimit*1.01
        else:
            buyExecute = bar.close * 1.02
            shortExecute = bar.close * 0.98
        return buyExecute, shortExecute

    def buyCheckExtend(self, bar, volume=None):
        symbol = bar.vtSymbol
        if not volume:
            volume = self.lot
        buyExecute, shortExecute = self.priceExecute(bar)
        if self.orderDict[symbol + '_OPEN']:
            self.writeCtaLog('haveOpenOrder_Pass')
        else:
            self.buyOpen(symbol, buyExecute, volume)

    def shortCheckExtend(self, bar, volume=None):
        symbol = bar.vtSymbol
        if not volume:
            volume = self.lot
        buyExecute, shortExecute = self.priceExecute(bar)
        if self.orderDict[symbol + '_OPEN']:
            self.writeCtaLog('haveOpenOrder_Pass')
        else:
            self.shortOpen(symbol, shortExecute, volume)

    def coverCheckExtend(self, bar):
        symbol = bar.vtSymbol
        buyExecute, shortExecute = self.priceExecute(bar)
        cancelled,canceling = self.cancelCloseOrder(bar)
        if cancelled:
            self.coverClose(symbol, buyExecute, self.ownPosDict[symbol + "_SHORT"])
        else:
            self.toExcuteOrdersID += 1
            self.toExcuteOrders[self.toExcuteOrdersID] = {
                "symbol": symbol,
                "price": buyExecute,
                "volume": self.ownPosDict[symbol + "_SHORT"],
                "orderType": "coverClose",
                "canceling":canceling
            }

    def sellCheckExtend(self, bar):
        symbol = bar.vtSymbol
        buyExecute, shortExecute = self.priceExecute(bar)
        cancelled,canceling = self.cancelCloseOrder(bar)
        if cancelled:
            self.sellClose(symbol, shortExecute, self.ownPosDict[symbol + '_LONG'])
        else:
            self.toExcuteOrdersID += 1
            self.toExcuteOrders[self.toExcuteOrdersID] = {
                "symbol": symbol,
                "price": shortExecute,
                "volume": self.ownPosDict[symbol + '_LONG'],
                "orderType": "sellClose",
                "canceling": canceling
            }

    def shortOpen(self, symbol, price, volume):
        shortOpenOrderList = self.short(symbol, price, volume)
        self.orderDict[symbol + '_OPEN'].extend(shortOpenOrderList)

    def buyOpen(self, symbol, price, volume):
        buyOpenOrderList = self.buy(symbol, price, volume)
        self.orderDict[symbol + '_OPEN'].extend(buyOpenOrderList)

    def sellClose(self, symbol, price, volume):
        sellCloseOrderList = self.sell(symbol, price, volume)
        self.orderDict[symbol + '_CLOSE'].extend(sellCloseOrderList)

    def coverClose(self, symbol, price, volume):
        coverCloseOrderList = self.cover(symbol, price, volume)
        self.orderDict[symbol + '_CLOSE'].extend(coverCloseOrderList)


# executeManagement--------------------------------------------------------
#     def on5sBar(self, bar):
    def onBar(self, bar):
        self.writeCtaLog('adxMacdEosStrategy####5S####ownposDict:%s####'%(self.ownPosDict))
        self.onBarRiskControl(bar)
        self.onBarExitTimeControl(bar)
        self.onBarExecute(bar)
        self.onBarPosition(bar)

    def onBarExit(self, bar):
        symbol = bar.vtSymbol
        if self.priceDirection == -1:
            if self.ownPosDict[symbol + "_LONG"] > 0 :
                self.sellCheckExtend(bar)
        elif self.priceDirection == 1:
            if self.ownPosDict[symbol + "_SHORT"] > 0:
                self.coverCheckExtend(bar)

    def onBarExitTimeControl(self, bar):
        if self.closeTime:
            if (bar.datetime - self.closeTime) >= timedelta(hours=self.stopControlTime):
                self.stopLossControl = 0

    def onBarRiskControl(self, bar):
        symbol = bar.vtSymbol
        if self.ownPosDict[symbol + "_LONG"] == 0 and self.ownPosDict[symbol + "_SHORT"] == 0:
            self.intraTradeHighDict = 0
            self.intraTradeLowDict = 999999
            self.longStop = 0
            self.shortStop = 999999
            self.nChange = 0

        # 持有多头仓位
        elif self.ownPosDict[symbol + "_LONG"] > 0:
            firstOrder=self.transactionPrice[symbol+'0']
            self.intraTradeHighDict = max(self.intraTradeHighDict, bar.high)
            self.nChange = (self.intraTradeHighDict/firstOrder-1)//self.trailingPct
            changePrice = firstOrder*self.nChange*self.trailingPct
            self.longStop = max(self.longStop, firstOrder*(1-self.trailingPct)+changePrice)
            if bar.low <= self.longStop:
                self.sellCheckExtend(bar)
            self.writeCtaLog('longStop%s'%(self.longStop))
        # 持有空头仓位
        elif self.ownPosDict[symbol + "_SHORT"] > 0:
            firstOrder=self.transactionPrice[symbol+'0']
            self.intraTradeLowDict = min(self.intraTradeLowDict, bar.low)
            self.nChange = -1* (self.intraTradeLowDict/firstOrder-1)//self.trailingPct
            changePrice = firstOrder*self.nChange*self.trailingPct
            self.shortStop = min(self.shortStop, firstOrder*(1+self.trailingPct)-changePrice)
            if bar.high >= self.shortStop:
                self.coverCheckExtend(bar)
            self.writeCtaLog('shortStop%s'%(self.shortStop))

    def onBarExecute(self, bar):
        symbol = bar.vtSymbol
        if (self.adxCanTrade == 1):
            if (self.priceDirection == 1) and (self.ownPosDict[symbol + "_LONG"] == 0):
                if self.stopLossControl == -1:
                    self.stopLossControl = 0
                if (self.stopLossControl == 0):
                    if (self.ownPosDict[symbol + "_SHORT"] == 0):
                        self.buyCheckExtend(bar)
                    elif (self.ownPosDict[symbol + "_SHORT"] > 0):
                        self.coverCheckExtend(bar)
                        self.buyCheckExtend(bar)

            elif (self.priceDirection == -1) and (self.ownPosDict[symbol + "_SHORT"] == 0):
                if self.stopLossControl == 1:
                    self.stopLossControl = 0
                if (self.stopLossControl == 0):
                    if (self.ownPosDict[symbol + "_LONG"] == 0):
                        self.shortCheckExtend(bar)
                    elif (self.ownPosDict[symbol + "_LONG"] > 0):
                        self.sellCheckExtend(bar)
                        self.shortCheckExtend(bar)
            else:
                self.onBarExit(bar)
        else:
            self.onBarExit(bar)
        self.putEvent()

    def onBarPosition(self,bar):
        symbol = bar.vtSymbol
        firstOrder=self.transactionPrice[symbol+'0']
        if (self.ownPosDict[symbol+'_LONG']==0) and (self.ownPosDict[symbol + "_SHORT"]==0):
            self.nPos=0
        elif (self.ownPosDict[symbol+'_LONG']>0 and self.nPos < 1):
            if (firstOrder/bar.close-1) >= self.addPct:
                self.nPos += 1
                self.buyCheckExtend(bar, int(self.lot*(self.addMultipler**self.nPos)))
        elif (self.ownPosDict[symbol + "_SHORT"] > 0 and self.nPos < 1):
            if (bar.close/firstOrder-1) >= self.addPct:
                self.nPos += 1
                self.shortCheckExtend(bar, int(self.lot*(self.addMultipler**self.nPos)))


    # ----------------------------------------------------------------------
 
    def on30MinBar(self, bar):
        symbol = bar.vtSymbol

        am30 = self.getArrayManager(symbol, "30m")

        if not am30.inited:
            return

        adxTrend = ta.ADX(am30.high, am30.low, am30.close, self.adxPeriod)
        VWAP = ta.SUM(am30.close*am30.volume, self.VWAPPeriod) / ta.SUM(am30.volume, self.VWAPPeriod)

# 现象条件
        breakUpVwap = (am30.close[-1] > VWAP[-2]) and (am30.close[-2] <= VWAP[-2])
        breakDnVwap = (am30.close[-1] < VWAP[-2]) and (am30.close[-2] >= VWAP[-2])
        more_volume=((am30.volume[-2]+am30.volume[-3]+am30.volume[-4])/3)>((am30.volume[-4]+am30.volume[-5]+am30.volume[-6])/3)
        HH=max(am30.close[-10:-2])
        LL=min(am30.low[-10:-2])
        mom=am30.close[-1]>am30.close[-self.long]
        if(more_volume) and (mom>0 or mom<0):
            self.adxCanTrade = 1
        else:
            self.adxCanTrade = -1

        self.writeCtaLog('adxTrend: %s, adxCanTrade: %s:'%(adxTrend[-1], self.adxCanTrade))

        macd, macdSignal, macdHist = ta.MACDEXT(am30.close, self.fastPeriod, self.fastMaType, \
                                                self.slowPeriod, self.slowMaType, self.signalPeriod, self.signalMaType)

        if (macd[-1]>macdSignal[-1]) and (macd[-1]>macd[-3]):
            self.priceDirection = 1
        elif (macd[-1]<macdSignal[-1]) and (macd[-1]<macd[-3]):
            self.priceDirection = -1
        else:
            self.priceDirection = 0


        #self.writeCtaLog(u'macd: %s, macdSignal: %s, priceDirection: %s'%(macd[-3:], macdSignal[-1], self.priceDirection))
        self.putEvent()

    # ----------------------------------------------------------------------
    def dealtoExcuteOrders(self, symbol):
        for ID in list(self.toExcuteOrders):
            order = self.toExcuteOrders[ID]
            if order["orderType"] in ["coverClose", "sellClose"]:
                # 前置要求中待撤销的订单已从orderDict中全部撤销
                if len(set(order["canceling"]) & set(self.orderDict[symbol + '_CLOSE'])) == 0:
                    if order["orderType"] == "coverClose":
                        self.coverClose(order["symbol"], order["price"], order["volume"])
                    elif order["orderType"] == "sellClose":
                        self.sellClose(order["symbol"], order["price"], order["volume"])
                    # 清除该待执行订单
                    del self.toExcuteOrders[ID]

    # ----------------------------------------------------------------------
    def onOrder(self, order):
        symbol = order.vtSymbol
        self.onOrderEmail(order)

        if order.status in STATUS_FINISHED:
            if (order.offset == OFFSET_OPEN) and (str(order.vtOrderID) in self.orderDict[symbol + '_OPEN']):
                self.orderDict[symbol + '_OPEN'].remove(str(order.vtOrderID))
            elif (order.offset == OFFSET_CLOSE) and (str(order.vtOrderID) in self.orderDict[symbol + '_CLOSE']):
                self.orderDict[symbol + '_CLOSE'].remove(str(order.vtOrderID))
        # 触发撤单成功,扫描待执行订单的前置撤单要求是否达到，达到则触发对待执行订单的发单
        if order.status == STATUS_CANCELLED:
            self.dealtoExcuteOrders(symbol)

    def onOrderEmail(self, order):
        if order.status == STATUS_UNKNOWN:
            self.mail(u'出现未知订单，需要策略师外部干预,ID:%s, symbol:%s,direction:%s,offset:%s'
                      % (order.vtOrderID, order.vtSymbol, order.direction, order.offset))
        if order.status == STATUS_REJECTED:
            self.mail(u'Rejected,ID:%s, symbol:%s,direction:%s,offset:%s,拒单信息:%s'
                      % (order.vtOrderID, order.vtSymbol, order.direction, order.offset,order.rejectedInfo))
        if order.thisTradedVolume != 0:
            content = u'成交信息播报,ID:%s, symbol:%s, directionL%s, offset:%s, price:%s'%\
                      (order.vtOrderID, order.vtSymbol, order.direction, order.offset, order.price)
            self.mail(content)

    # ----------------------------------------------------------------------
    def onTrade(self, trade):
        symbol = trade.vtSymbol
        self.writeCtaLog('tradeTime:%s,offset:%s,transactionPrice:%s ,ownPosDict%s'\
                        %(trade.tradeDatetime, trade.offset, self.transactionPrice, self.ownPosDict))
        if trade.offset == OFFSET_OPEN:
            self.transactionPrice[symbol+'%s'%(self.nPos)] = trade.price
        elif trade.offset == OFFSET_CLOSE:
            self.closeTime = trade.tradeDatetime
            if trade.direction == DIRECTION_SHORT:
                self.stopLossControl = 1
            elif trade.direction == DIRECTION_LONG:
                self.stopLossControl = -1
        self.onTradeOwnPosDict(trade)
        self.takeProfit(trade)

    def onTradeOwnPosDict(self, trade):
        symbol = trade.vtSymbol
        if trade.direction == DIRECTION_LONG and trade.offset == OFFSET_OPEN:
            self.ownPosDict[symbol + '_LONG'] += int(trade.volume)
        elif trade.direction == DIRECTION_SHORT and trade.offset == OFFSET_CLOSE:
            self.ownPosDict[symbol + '_LONG'] -= int(trade.volume)
        elif trade.direction == DIRECTION_SHORT and trade.offset == OFFSET_OPEN:
            self.ownPosDict[symbol + '_SHORT'] += int(trade.volume)
        elif trade.direction == DIRECTION_LONG and trade.offset == OFFSET_CLOSE:
            self.ownPosDict[symbol + '_SHORT'] -= int(trade.volume)

    def takeProfit(self, trade):
        symbol = trade.vtSymbol
        firstOrder = self.transactionPrice[symbol+'0']
        longFirstProfit = firstOrder*(1+self.takeProfitFirstPct)
        longSecondProfit = firstOrder*(1+self.takeProfitSecondPct)
        shortFirstProfit = firstOrder*(1-self.takeProfitFirstPct)
        shortSecondProfit = firstOrder*(1-self.takeProfitSecondPct)

        if trade.offset == OFFSET_OPEN:
            if trade.direction == DIRECTION_LONG:
                self.sellClose(symbol, longFirstProfit, trade.volume//2)
                self.sellClose(symbol, longSecondProfit, trade.volume-trade.volume//2)
                self.writeCtaLog('long### tp1:%s, tp2:%s'%(longFirstProfit, longSecondProfit))
            elif trade.direction == DIRECTION_SHORT:
                self.coverClose(symbol, shortFirstProfit, trade.volume//2)
                self.coverClose(symbol, shortSecondProfit,trade.volume-trade.volume//2)
                self.writeCtaLog('short### tp1:%s, tp2:%s'%(shortFirstProfit, shortSecondProfit))
    # ----------------------------------------------------------------------
    def onStopOrder(self, so):
        """停止单推送"""
        pass

In [ ]:
from vnpy.trader.app.ctaStrategy import BacktestingEngine
import pandas as pd
from vnpy.trader.utils import htmlplot

def runBacktesting(strategyClass, settingDict,
                   startDate, endDate, slippage, rate):
    engine = BacktestingEngine()
    engine.setBacktestingMode(engine.BAR_MODE)
    engine.setDatabase('VnTrader_1Min_Db')
    engine.setStartDate(startDate, initHours=100)
    engine.setEndDate(endDate)
    engine.setSlippage(slippage)
    engine.setRate(rate)
    engine.setLog(True)
    
#   在这里设置画图参数，目前只用设置freqency，支持的有 s(second) m(minute) h(hour) d(day)，也可以用timestamp, 例如： 1h -> timestamp(hours=1)
#   sample(输出用4hbar画图)
#   画图需要安装bokeh==0.12.14
    engine.initStrategy(strategyClass, settingDict)
    engine.setCapital(200000)
    engine.runBacktesting()
    #显示逐日回测结果
    engine.showDailyResult()
    #显示逐笔回测结果
    engine.showBacktestingResult()
    htmlplot.showTransaction(engine, frequency="1h")
    # 计算回测结果
    perfromance = engine.calculateDailyResult()
    perfromanceDf , result = engine.calculateDailyStatistics(perfromance)
    tradeReport = pd.DataFrame([obj.__dict__ for obj in engine.tradeDict.values()])
    tradeDf = tradeReport.set_index('tradeDatetime')
    return engine
#     return perfromanceDf, tradeDf

runBacktesting(adxMacdEosStrategy, {'symbolList': [
                                                       # 'EOSUSDT:binance',
                                                             'eos_quarter:OKEX'
                                                        ]},
                                                           '20181001 12:00', 
#                                                                '20181006 12:00', 
                                                           '20190110 16:00',0.002, 5/10000)

仓位字典构造完成 
初始仓位: {'eos_quarter:OKEX_LONG': 0, 'eos_quarter:OKEX_SHORT': 0}
2019-01-19 18:07:24.735713	开始回测
2019-01-19 18:07:24.735914	策略初始化
2019-01-19 18:07:25.188165	数据载入完成, 时间段:[20180927 08:00,20181001 12:00);数据量:5266
2019-01-19 18:07:25.188447	初始化预加载数据成功, 数据长度:5266
2019-01-19 18:07:25.188557	策略初始化完成
2019-01-19 18:07:25.188598	策略启动完成
2019-01-19 18:07:25.188666	回测时间范围:[20181001 12:00,20190110 16:00)
2019-01-19 18:07:25.188743	当前回放的时间段:[20181001 12:00,20190110 16:00)
2019-01-19 18:07:35.844131	数据载入完成, 时间段:[20181001 12:00,20190110 16:00);数据量:144434
2019-01-19 18:07:36.189149	数据载入完成, 时间段:[20180930 05:51,20181003 12:01);数据量:4689
2019-01-19 18:07:39.322969	数据载入完成, 时间段:[20180824 19:30,20180930 05:50);数据量:38886
2019-01-19 18:07:39.329037	WARNING: 该时间段:[20181003 12:01,20181003 12:30) 数据量为0!
2019-01-19 18:08:04.963831	回放结束ress: 100%    


In [10]:
import numpy as np
ara=[80,11,12,22,20,41,32,55,8,9]
HH=max(ara[-11:-1])
HH


80

In [72]:
if __name__=="__main__":
    from vnpy.trader.app.ctaStrategy import BacktestingEngine

    # 创建回测引擎对象
    engine = BacktestingEngine()

    # 设置回测使用的数据
    engine.setBacktestingMode(engine.BAR_MODE)    # 设置引擎的回测模式为K线
    engine.setDatabase('VnTrader_1Min_Db')  # 设置使用的历史数据库
    engine.setStartDate('20180701 12:00',initHours=200) # 设置回测用的数据起始日期
    engine.setEndDate('20181201 12:00') # 设置回测用的数据终止日期

    # 配置回测引擎参数
    engine.setSlippage(0.002)    # 设置滑点
    engine.setRate(5/10000)   # 设置手续费千1
    engine.setCapital(200000)  # 设置回测本金

    # # 在引擎中创建策略对象
    parameterDict = {'symbolList':['EOSUSDT:binance']}          # 策略参数配置
    engine.initStrategy(adxMacdEosStrategy, parameterDict)    # 创建策略对象
    engine.runBacktesting()

仓位字典构造完成 
初始仓位: {'EOSUSDT:binance_LONG': 0, 'EOSUSDT:binance_SHORT': 0}
2019-01-19 15:01:14.748929	开始回测
2019-01-19 15:01:14.749934	策略初始化
2019-01-19 15:01:15.321398	数据载入完成, 时间段:[20180623 04:00,20180701 12:00);数据量:11295
2019-01-19 15:01:15.321398	初始化预加载数据成功, 数据长度:11295
2019-01-19 15:01:15.321398	策略初始化完成
2019-01-19 15:01:15.321398	策略启动完成
2019-01-19 15:01:15.322396	回测时间范围:[20180701 12:00,20181201 12:00)
2019-01-19 15:01:15.322396	当前回放的时间段:[20180701 12:00,20181201 12:00)
2019-01-19 15:01:56.332076	MongoDB无法连接,当前仅使用本地缓存数据, 请注意数据量。
2019-01-19 15:01:56.407780	数据载入完成, 时间段:[20180701 12:00,20181201 12:00);数据量:218513


Traceback (most recent call last):
  File "C:\Users\peng\Anaconda3\lib\site-packages\vnpy_fxdayu-1.1.17-py3.6.egg\vnpy\trader\app\ctaStrategy\ctaBacktesting.py", line 285, in loadHistoryData
    if symbol in self.dbClient[self.dbName].collection_names():
  File "C:\Users\peng\Anaconda3\lib\site-packages\pymongo\database.py", line 715, in collection_names
    nameOnly=True, **kws)]
  File "C:\Users\peng\Anaconda3\lib\site-packages\pymongo\database.py", line 674, in list_collections
    read_pref) as (sock_info, slave_okay):
  File "C:\Users\peng\Anaconda3\lib\contextlib.py", line 81, in __enter__
    return next(self.gen)
  File "C:\Users\peng\Anaconda3\lib\site-packages\pymongo\mongo_client.py", line 1099, in _socket_for_reads
    server = topology.select_server(read_preference)
  File "C:\Users\peng\Anaconda3\lib\site-packages\pymongo\topology.py", line 224, in select_server
    address))
  File "C:\Users\peng\Anaconda3\lib\site-packages\pymongo\topology.py", line 183, in select_serve

2019-01-19 15:01:56.649106	数据载入完成, 时间段:[20180630 05:51,20180703 12:01);数据量:4690
2019-01-19 15:02:29.270624	MongoDB无法连接,当前仅使用本地缓存数据, 请注意数据量。
2019-01-19 15:02:29.284489	数据载入完成, 时间段:[20180524 19:30,20180630 05:50);数据量:46385
2019-01-19 15:02:29.292438	WARNING: 该时间段:[20180703 12:01,20180703 12:30) 数据量为0!


Traceback (most recent call last):
  File "C:\Users\peng\Anaconda3\lib\site-packages\vnpy_fxdayu-1.1.17-py3.6.egg\vnpy\trader\app\ctaStrategy\ctaBacktesting.py", line 285, in loadHistoryData
    if symbol in self.dbClient[self.dbName].collection_names():
  File "C:\Users\peng\Anaconda3\lib\site-packages\pymongo\database.py", line 715, in collection_names
    nameOnly=True, **kws)]
  File "C:\Users\peng\Anaconda3\lib\site-packages\pymongo\database.py", line 674, in list_collections
    read_pref) as (sock_info, slave_okay):
  File "C:\Users\peng\Anaconda3\lib\contextlib.py", line 81, in __enter__
    return next(self.gen)
  File "C:\Users\peng\Anaconda3\lib\site-packages\pymongo\mongo_client.py", line 1099, in _socket_for_reads
    server = topology.select_server(read_preference)
  File "C:\Users\peng\Anaconda3\lib\site-packages\pymongo\topology.py", line 224, in select_server
    address))
  File "C:\Users\peng\Anaconda3\lib\site-packages\pymongo\topology.py", line 183, in select_serve

2019-01-19 15:03:05.480711	回放结束ress: 100%    


In [74]:
from vnpy.trader.app.ctaStrategy.ctaBacktesting  import OptimizationSetting
# 优化配置
from vnpy.trader.app.ctaStrategy import BacktestingEngine
import pandas as pd
setting = OptimizationSetting()                # 新建一个优化任务设置对象
setting.setOptimizeTarget('sharpeRatio')        # 设置优化排序的目标是夏普
setting.addParameter('fastPeriod', 20, 40, 10)    # 增加第一个优化参数，起始30，结束50，步进10
setting.addParameter('slowPeriod', 40, 80, 20)    # 增加第二个优化参数，起始60，结束30，步进10
setting.addParameter('signalMaPeriod', 10, 20, 5)    # 增加第二个优化参数，起始10，结束30，步进5
#setting.addParameter('long', 8, 24, 2)
setting.addParameter('symbolList', ['EOSUSDT:binance']) 

import time
start = time.time()
# 执行单线程优化
resultList = engine.runOptimization(adxMacdEosStrategy, setting)
# 执行多进程优化一般会改写成py文件然后用多线程运行，提高优化速度。
# resultList = engine.runParallelOptimization(MultiFrameMaStrategy, setting)
print('耗时：%s' %(time.time()-start))

2019-01-19 15:27:52.997067	------------------------------
2019-01-19 15:27:52.998068	setting: {'fastPeriod': 20, 'slowPeriod': 40, 'signalMaPeriod': 10, 'symbolList': ['EOSUSDT:binance']}
仓位字典构造完成 
初始仓位: {'EOSUSDT:binance_LONG': 0, 'EOSUSDT:binance_SHORT': 0}
2019-01-19 15:27:53.050924	开始回测
2019-01-19 15:27:53.050924	策略初始化
2019-01-19 15:27:53.618407	数据载入完成, 时间段:[20180623 04:00,20180701 12:00);数据量:11295
2019-01-19 15:27:53.618407	初始化预加载数据成功, 数据长度:11295
2019-01-19 15:27:53.618407	策略初始化完成
2019-01-19 15:27:53.618407	策略启动完成
2019-01-19 15:27:53.618407	回测时间范围:[20180701 12:00,20181201 12:00)
2019-01-19 15:27:53.618407	当前回放的时间段:[20180701 12:00,20181201 12:00)
2019-01-19 15:28:33.776910	MongoDB无法连接,当前仅使用本地缓存数据, 请注意数据量。
2019-01-19 15:28:33.855578	数据载入完成, 时间段:[20180701 12:00,20181201 12:00);数据量:218513


Traceback (most recent call last):
  File "C:\Users\peng\Anaconda3\lib\site-packages\vnpy_fxdayu-1.1.17-py3.6.egg\vnpy\trader\app\ctaStrategy\ctaBacktesting.py", line 285, in loadHistoryData
    if symbol in self.dbClient[self.dbName].collection_names():
  File "C:\Users\peng\Anaconda3\lib\site-packages\pymongo\database.py", line 715, in collection_names
    nameOnly=True, **kws)]
  File "C:\Users\peng\Anaconda3\lib\site-packages\pymongo\database.py", line 674, in list_collections
    read_pref) as (sock_info, slave_okay):
  File "C:\Users\peng\Anaconda3\lib\contextlib.py", line 81, in __enter__
    return next(self.gen)
  File "C:\Users\peng\Anaconda3\lib\site-packages\pymongo\mongo_client.py", line 1099, in _socket_for_reads
    server = topology.select_server(read_preference)
  File "C:\Users\peng\Anaconda3\lib\site-packages\pymongo\topology.py", line 224, in select_server
    address))
  File "C:\Users\peng\Anaconda3\lib\site-packages\pymongo\topology.py", line 183, in select_serve

2019-01-19 15:28:34.116880	数据载入完成, 时间段:[20180630 05:51,20180703 12:01);数据量:4690
2019-01-19 15:29:06.566372	MongoDB无法连接,当前仅使用本地缓存数据, 请注意数据量。
2019-01-19 15:29:06.582201	数据载入完成, 时间段:[20180524 19:30,20180630 05:50);数据量:46385
2019-01-19 15:29:06.591177	WARNING: 该时间段:[20180703 12:01,20180703 12:30) 数据量为0!


Traceback (most recent call last):
  File "C:\Users\peng\Anaconda3\lib\site-packages\vnpy_fxdayu-1.1.17-py3.6.egg\vnpy\trader\app\ctaStrategy\ctaBacktesting.py", line 285, in loadHistoryData
    if symbol in self.dbClient[self.dbName].collection_names():
  File "C:\Users\peng\Anaconda3\lib\site-packages\pymongo\database.py", line 715, in collection_names
    nameOnly=True, **kws)]
  File "C:\Users\peng\Anaconda3\lib\site-packages\pymongo\database.py", line 674, in list_collections
    read_pref) as (sock_info, slave_okay):
  File "C:\Users\peng\Anaconda3\lib\contextlib.py", line 81, in __enter__
    return next(self.gen)
  File "C:\Users\peng\Anaconda3\lib\site-packages\pymongo\mongo_client.py", line 1099, in _socket_for_reads
    server = topology.select_server(read_preference)
  File "C:\Users\peng\Anaconda3\lib\site-packages\pymongo\topology.py", line 224, in select_server
    address))
  File "C:\Users\peng\Anaconda3\lib\site-packages\pymongo\topology.py", line 183, in select_serve

2019-01-19 15:29:41.210649	回放结束ress: 100%    
2019-01-19 15:29:41.210649	计算按日统计结果
2019-01-19 15:29:41.342290	------------------------------
2019-01-19 15:29:41.342290	setting: {'fastPeriod': 20, 'slowPeriod': 40, 'signalMaPeriod': 15, 'symbolList': ['EOSUSDT:binance']}
仓位字典构造完成 
初始仓位: {'EOSUSDT:binance_LONG': 0, 'EOSUSDT:binance_SHORT': 0}
2019-01-19 15:29:41.365230	开始回测
2019-01-19 15:29:41.365230	策略初始化
2019-01-19 15:29:41.909774	数据载入完成, 时间段:[20180623 04:00,20180701 12:00);数据量:11295
2019-01-19 15:29:41.909774	初始化预加载数据成功, 数据长度:11295
2019-01-19 15:29:41.909774	策略初始化完成
2019-01-19 15:29:41.909774	策略启动完成
2019-01-19 15:29:41.909774	回测时间范围:[20180701 12:00,20181201 12:00)
2019-01-19 15:29:41.909774	当前回放的时间段:[20180701 12:00,20181201 12:00)
2019-01-19 15:30:21.882857	MongoDB无法连接,当前仅使用本地缓存数据, 请注意数据量。
2019-01-19 15:30:21.989569	数据载入完成, 时间段:[20180701 12:00,20181201 12:00);数据量:218513


...

2019-01-19 16:24:47.174102	回放结束ress: 100%    
2019-01-19 16:24:47.175101	计算按日统计结果
2019-01-19 16:24:47.362598	------------------------------
2019-01-19 16:24:47.362598	setting: {'fastPeriod': 40, 'slowPeriod': 80, 'signalMaPeriod': 20, 'symbolList': ['EOSUSDT:binance']}
仓位字典构造完成 
初始仓位: {'EOSUSDT:binance_LONG': 0, 'EOSUSDT:binance_SHORT': 0}
2019-01-19 16:24:47.396507	开始回测
2019-01-19 16:24:47.396507	策略初始化
2019-01-19 16:24:48.006876	数据载入完成, 时间段:[20180623 04:00,20180701 12:00);数据量:11295
2019-01-19 16:24:48.006876	初始化预加载数据成功, 数据长度:11295
2019-01-19 16:24:48.006876	策略初始化完成
2019-01-19 16:24:48.006876	策略启动完成
2019-01-19 16:24:48.007875	回测时间范围:[20180701 12:00,20181201 12:00)
2019-01-19 16:24:48.007875	当前回放的时间段:[20180701 12:00,20181201 12:00)
2019-01-19 16:25:30.454419	MongoDB无法连接,当前仅使用本地缓存数据, 请注意数据量。
2019-01-19 16:25:30.523234	数据载入完成, 时间段:[20180701 12:00,20181201 12:00);数据量:218513


Traceback (most recent call last):
  File "C:\Users\peng\Anaconda3\lib\site-packages\vnpy_fxdayu-1.1.17-py3.6.egg\vnpy\trader\app\ctaStrategy\ctaBacktesting.py", line 285, in loadHistoryData
    if symbol in self.dbClient[self.dbName].collection_names():
  File "C:\Users\peng\Anaconda3\lib\site-packages\pymongo\database.py", line 715, in collection_names
    nameOnly=True, **kws)]
  File "C:\Users\peng\Anaconda3\lib\site-packages\pymongo\database.py", line 674, in list_collections
    read_pref) as (sock_info, slave_okay):
  File "C:\Users\peng\Anaconda3\lib\contextlib.py", line 81, in __enter__
    return next(self.gen)
  File "C:\Users\peng\Anaconda3\lib\site-packages\pymongo\mongo_client.py", line 1099, in _socket_for_reads
    server = topology.select_server(read_preference)
  File "C:\Users\peng\Anaconda3\lib\site-packages\pymongo\topology.py", line 224, in select_server
    address))
  File "C:\Users\peng\Anaconda3\lib\site-packages\pymongo\topology.py", line 183, in select_serve

2019-01-19 16:25:30.772568	数据载入完成, 时间段:[20180630 05:51,20180703 12:01);数据量:4690
2019-01-19 16:27:55.448983	MongoDB无法连接,当前仅使用本地缓存数据, 请注意数据量。
2019-01-19 16:27:55.494771	数据载入完成, 时间段:[20180524 19:30,20180630 05:50);数据量:46385
2019-01-19 16:27:55.510698	WARNING: 该时间段:[20180703 12:01,20180703 12:30) 数据量为0!


Traceback (most recent call last):
  File "C:\Users\peng\Anaconda3\lib\site-packages\vnpy_fxdayu-1.1.17-py3.6.egg\vnpy\trader\app\ctaStrategy\ctaBacktesting.py", line 285, in loadHistoryData
    if symbol in self.dbClient[self.dbName].collection_names():
  File "C:\Users\peng\Anaconda3\lib\site-packages\pymongo\database.py", line 715, in collection_names
    nameOnly=True, **kws)]
  File "C:\Users\peng\Anaconda3\lib\site-packages\pymongo\database.py", line 674, in list_collections
    read_pref) as (sock_info, slave_okay):
  File "C:\Users\peng\Anaconda3\lib\contextlib.py", line 81, in __enter__
    return next(self.gen)
  File "C:\Users\peng\Anaconda3\lib\site-packages\pymongo\mongo_client.py", line 1099, in _socket_for_reads
    server = topology.select_server(read_preference)
  File "C:\Users\peng\Anaconda3\lib\site-packages\pymongo\topology.py", line 224, in select_server
    address))
  File "C:\Users\peng\Anaconda3\lib\site-packages\pymongo\topology.py", line 183, in select_serve

2019-01-19 16:28:32.206613	回放结束ress: 100%    
2019-01-19 16:28:32.206613	计算按日统计结果
2019-01-19 16:28:32.270442	------------------------------
2019-01-19 16:28:32.270442	优化结果：
2019-01-19 16:28:32.270442	参数：["{'fastPeriod': 40, 'slowPeriod': 60, 'signalMaPeriod': 10, 'symbolList': ['EOSUSDT:binance']}"]，目标：1.03288683639
2019-01-19 16:28:32.270442	参数：["{'fastPeriod': 40, 'slowPeriod': 60, 'signalMaPeriod': 15, 'symbolList': ['EOSUSDT:binance']}"]，目标：1.03288683639
2019-01-19 16:28:32.270442	参数：["{'fastPeriod': 40, 'slowPeriod': 60, 'signalMaPeriod': 20, 'symbolList': ['EOSUSDT:binance']}"]，目标：1.03288683639
2019-01-19 16:28:32.270442	参数：["{'fastPeriod': 30, 'slowPeriod': 80, 'signalMaPeriod': 10, 'symbolList': ['EOSUSDT:binance']}"]，目标：0.884769784575
2019-01-19 16:28:32.270442	参数：["{'fastPeriod': 30, 'slowPeriod': 80, 'signalMaPeriod': 15, 'symbolList': ['EOSUSDT:binance']}"]，目标：0.884769784575
2019-01-19 16:28:32.270442	参数：["{'fastPeriod': 30, 'slowPeriod': 80, 'signalMaPeriod': 20, 'symbolLi

In [486]:
####参数优化

In [1]:
"""from vnpy.trader.app.ctaStrategy.ctaBacktesting  import OptimizationSetting
# 优化配置
setting = OptimizationSetting()                # 新建一个优化任务设置对象
setting.setOptimizeTarget('sharpeRatio')        # 设置优化排序的目标是夏普
setting.addParameter("a", 1.00, 1.10, 0.01)    # 增加第一个优化参数，起始30，结束50，步进10
setting.addParameter("b", 0.90, 1.00, 0.01)    # 增加第二个优化参数，起始60，结束30，步进10
# setting.addParameter('signalMaPeriod', 10, 20, 5)    # 增加第二个优化参数，起始10，结束30，步进5
setting.addParameter('symbolList', ['BTCUSDT:binance']) 
import time
start = time.time()
# 执行单线程优化
resultList = engine.runOptimization(MACDRSIStrategy, setting)
print('耗时：%s' %(time.time()-start))"""

'from vnpy.trader.app.ctaStrategy.ctaBacktesting  import OptimizationSetting\n# 优化配置\nsetting = OptimizationSetting()                # 新建一个优化任务设置对象\nsetting.setOptimizeTarget(\'sharpeRatio\')        # 设置优化排序的目标是夏普\nsetting.addParameter("a", 1.00, 1.10, 0.01)    # 增加第一个优化参数，起始30，结束50，步进10\nsetting.addParameter("b", 0.90, 1.00, 0.01)    # 增加第二个优化参数，起始60，结束30，步进10\n# setting.addParameter(\'signalMaPeriod\', 10, 20, 5)    # 增加第二个优化参数，起始10，结束30，步进5\nsetting.addParameter(\'symbolList\', [\'BTCUSDT:binance\']) \nimport time\nstart = time.time()\n# 执行单线程优化\nresultList = engine.runOptimization(MACDRSIStrategy, setting)\nprint(\'耗时：%s\' %(time.time()-start))'

In [3]:
#import pandas as pd
#print(pd.DataFrame(resultList).sort_values(1,  ascending=False))